<a href="https://colab.research.google.com/github/Shizakhan99/Project4-Team1/blob/main/Stroke_Sara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# Read salary data
file_path = Path("/content/drive/MyDrive/Colab Notebooks/Resources/Stroke_dataset.csv")
df_stroke = pd.read_csv(file_path)

# Display sample data
df_stroke.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [ ]:
df_stroke.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


In [ ]:
df_stroke["bmi"].isnull

<bound method Series.isnull of 0       36.6
1        NaN
2       32.5
3       34.4
4       24.0
        ... 
5105     NaN
5106    40.0
5107    30.6
5108    25.6
5109    26.2
Name: bmi, Length: 5110, dtype: float64>

In [ ]:
df_stroke_cleaned = df_stroke.dropna(subset=['bmi'])
df_stroke_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4909 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4909 non-null   int64  
 1   gender             4909 non-null   object 
 2   age                4909 non-null   float64
 3   hypertension       4909 non-null   int64  
 4   heart_disease      4909 non-null   int64  
 5   ever_married       4909 non-null   object 
 6   work_type          4909 non-null   object 
 7   Residence_type     4909 non-null   object 
 8   avg_glucose_level  4909 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     4909 non-null   object 
 11  stroke             4909 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 498.6+ KB




```
# This is formatted as code
```

## Preprocess categorical data

## Artificial Neural Networks Based Optimization

In [ ]:
X = df_stroke_cleaned.drop(columns=['stroke'], inplace=False)
y = df_stroke_cleaned['stroke']

In [ ]:
X = pd.get_dummies(X)
X.head()
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4909 entries, 0 to 5109
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              4909 non-null   int64  
 1   age                             4909 non-null   float64
 2   hypertension                    4909 non-null   int64  
 3   heart_disease                   4909 non-null   int64  
 4   avg_glucose_level               4909 non-null   float64
 5   bmi                             4909 non-null   float64
 6   gender_Female                   4909 non-null   bool   
 7   gender_Male                     4909 non-null   bool   
 8   gender_Other                    4909 non-null   bool   
 9   ever_married_No                 4909 non-null   bool   
 10  ever_married_Yes                4909 non-null   bool   
 11  work_type_Govt_job              4909 non-null   bool   
 12  work_type_Never_worked          4909 no

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
print(X_train_scaled.shape)


(3927, 22)


## Compile, Train and Evaluate the Model

In [ ]:
import tensorflow as tf
!pip install keras-tuner --upgrade
import keras_tuner as kt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [ ]:

# Define the model creation function
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid'])

    # Allow kerastuner to decide number of neurons in the first layer
    nn_model.add(tf.keras.layers.Dense(
        units=hp.Int('first_units', min_value=1, max_value=12, step=2),
        activation=activation,
        input_dim=22
    ))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(
            units=hp.Int('units_' + str(i), min_value=1, max_value=10, step=2),
            activation=activation
        ))

    # Add output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

# Instantiate the tuner
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    directory="my_dir",
    project_name="tune_model"
)

# Run the kerastuner search for best hyperparameters
fit_model=tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

# Display the search results
tuner.results_summary()

Trial 60 Complete [00h 00m 16s]
val_accuracy: 0.9613034725189209

Best val_accuracy So Far: 0.9623218178749084
Total elapsed time: 00h 09m 15s
Results summary
Results in my_dir/tune_model
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0024 summary
Hyperparameters:
activation: tanh
first_units: 9
num_layers: 1
units_0: 3
units_1: 3
units_2: 1
units_3: 1
units_4: 3
units_5: 7
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0018
Score: 0.9623218178749084

Trial 0047 summary
Hyperparameters:
activation: tanh
first_units: 9
num_layers: 5
units_0: 3
units_1: 3
units_2: 1
units_3: 9
units_4: 9
units_5: 1
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0043
Score: 0.9623218178749084

Trial 0055 summary
Hyperparameters:
activation: relu
first_units: 11
num_layers: 1
units_0: 5
units_1: 7
units_2: 5
units_3: 7
units_4: 7
units_5: 3
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 9,
 'num_layers': 1,
 'units_0': 3,
 'units_1': 3,
 'units_2': 1,
 'units_3': 1,
 'units_4': 3,
 'units_5': 7,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0018'}

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

31/31 - 0s - 10ms/step - accuracy: 0.9623 - loss: 0.1488
Loss: 0.14881062507629395, Accuracy: 0.9623218178749084


In [ ]:
predictions = best_model.predict(X_test_scaled)

# Convert predictions to binary or class labels (depending on your model output)
# Assuming a binary classification model with a sigmoid activation
predicted_classes = (predictions > 0.5).astype(int)



# Generate a classification report
target_names = ["Class 0", "Class 1"]  # Update with your actual class names
print(classification_report(y_test, predicted_classes, target_names=target_names))

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

     Class 0       0.96      1.00      0.98       944
     Class 1       1.00      0.03      0.05        38

    accuracy                           0.96       982
   macro avg       0.98      0.51      0.52       982
weighted avg       0.96      0.96      0.94       982



In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd

# Calculate the confusion matrix
cm = confusion_matrix(y_test, predicted_classes)  # Ensure `predicted_classes` are processed as per the earlier snippet

# Create a DataFrame for the confusion matrix
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Print the confusion matrix
print("Confusion Matrix:")
print(cm_df)

# Calculate the accuracy score
acc_score = accuracy_score(y_test, predicted_classes)
print(f"Accuracy Score: {acc_score:.2f}")

Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0          944            0
Actual 1           37            1
Accuracy Score: 0.96
